일반화 성능도 있기 때문에 전체 데이터셋에 대해서 성능 평가를 해봐야 함

In [46]:
import pickle
# with open('../models/SVC (C=0.1)l30_r15.pkl', 'rb') as f:
with open('../models/SVC (C=0.1) norml30_r15.pkl', 'rb') as f:
    classifier = pickle.load(f)
    
from py.utils import load_data
head = 'l30_r15'
directory = '../data/'
x, y, x_words, vocabs = load_data(head, directory)
x = x.tocsr()

x shape = (15715, 4551)
y shape = (15715,)
# features = 4551
# L words = 15715


In [25]:
def get_sample_r(li, topk=5):
    nonzero = x[li,:].nonzero()[1]
    base = min(50, len(nonzero)//2)
    return [vocabs[r] for r in sorted(nonzero, key=lambda x:word_frequency[0,x], reverse=True)[base:base+topk]]

word_frequency = x.sum(axis=0)
word_frequency

matrix([[  48,   35,   29, ..., 2195,   17,   16]], dtype=int64)

In [48]:
# sparse matrix type
print(classifier.dual_coef_.shape)

# sum = 0 ?? alpha sum은 1아닌가?
print(classifier.dual_coef_.data.sum())

(1, 6282)
0.0


In [49]:
# Support vectors
classifier.support_vectors_.shape

(6282, 4551)

In [50]:
# Vector row id
classifier.support_

array([    1,     3,     6, ..., 15708, 15710, 15712], dtype=int32)

In [52]:
base = 0
topk = 40
for idx, l in sorted(enumerate(classifier.support_), key=lambda x:abs(classifier.dual_coef_[0,x[0]]), reverse=True)[base :base + topk]:
    print('%s%s - %s (%.3f)' % (x_words[l], '' if y[l] == 1 else '(다)', get_sample_r(l), classifier.dual_coef_[0,idx]))

끓(다) - ['도록', '었던', '기도', '었습니다', '는가'] (-0.100)
무서(다) - ['우면서도', '우니까', '워하는', '워져', '니께'] (-0.100)
구겨(다) - ['댔다', '져서', '졌고', '놓았다', '넣고'] (-0.100)
두들겨(다) - ['져서', '봐야', '대고', '대며', '보면서'] (-0.100)
짚(다) - ['어놓고', '어낸', '어야만', '였을', '음으로써'] (-0.100)
두들기(다) - ['자', '라고', '다가', '니', '거나'] (-0.100)
이어진(다) - ['다고', '다면', '데다'] (-0.100)
씌우(다) - ['기도', '거나', '죠', '려고', '냐'] (-0.100)
비해(다) - ['선', '볼', '보면', '서나'] (-0.100)
얼(다) - ['었어요', '기에는', '었지', '었는지', '어가는'] (-0.100)
웃겼(다) - ['나', '어요', '는지', '잖아', '듯이'] (-0.100)
찢겨(다) - ['버린', '가고', '버리고', '져서', '나가는'] (-0.100)
안기(다) - ['기도', '거나', '는가', '려고', '더니'] (-0.100)
뒤엎(다) - ['듯', '기를', '듯이', '기는', '겠다는'] (-0.100)
친해(다) - ['졌고', '지자', '졌습니다', '지기도', '진다는'] (-0.100)
저(다) - ['는지도', '으려는', '어라', '으라고', '어주는'] (-0.100)
부둥켜안(다) - ['는다', '다가', '구', '더니', '듯'] (-0.100)
깔깔거리(다) - ['던', '며', '다가', '거나'] (-0.100)
맡겨(다) - ['졌고', '왔던', '달라는', '지자', '졌습니다'] (-0.100)
매서(다) - ['움을', '운데', '웠고', '웠어'] (-0.100)
가져와(다) - ['요', '야지', '봐', '야겠다고', '봐라'] (-0.100)


In [53]:
from collections import Counter
alpha_count = Counter(classifier.dual_coef_.data)
print('number of alphas = %d' % len(alpha_count))
print('number of support vectors = %d\n' % len(classifier.dual_coef_.data))

for alpha, count in sorted(alpha_count.items(), key=lambda x:(x[1], abs(x[0])), reverse=True)[:50]:
    print('alpha= %.3f, count=%d (%.3f)' % (alpha, count, count/classifier.n_support_.sum()))

number of alphas = 2
number of support vectors = 6282

alpha= -0.100, count=3141 (0.500)
alpha= 0.100, count=3141 (0.500)


In [54]:
classifier.n_support_

array([3141, 3141], dtype=int32)